# 第 3 章: 正規表現


In [16]:
from pathlib import Path
import json
import re

### 20. JSON データの読み込み


In [10]:
def load_jsonlines(path: Path)->list[dict]:
    json_dicts = []
    with open(path, "r") as jsonl_file:
        for jsonline in jsonl_file:
            json_dicts.append(json.loads(jsonline))
    return json_dicts

wikis = load_jsonlines(Path("data/jawiki-country.json"))

In [118]:
uk_wikis = [wiki for wiki in wikis if wiki['title'] == "イギリス"]
uk_wiki = uk_wikis[0]
print(uk_wiki['text'][:5000])

{{redirect|UK}}
{{redirect|英国|春秋時代の諸侯国|英 (春秋)}}
{{Otheruses|ヨーロッパの国|長崎県・熊本県の郷土料理|いぎりす}}
{{基礎情報 国
|略名  =イギリス
|日本語国名 = グレートブリテン及び北アイルランド連合王国
|公式国名 = {{lang|en|United Kingdom of Great Britain and Northern Ireland}}<ref>英語以外での正式国名:<br />
*{{lang|gd|An Rìoghachd Aonaichte na Breatainn Mhòr agus Eirinn mu Thuath}}（[[スコットランド・ゲール語]]）
*{{lang|cy|Teyrnas Gyfunol Prydain Fawr a Gogledd Iwerddon}}（[[ウェールズ語]]）
*{{lang|ga|Ríocht Aontaithe na Breataine Móire agus Tuaisceart na hÉireann}}（[[アイルランド語]]）
*{{lang|kw|An Rywvaneth Unys a Vreten Veur hag Iwerdhon Glédh}}（[[コーンウォール語]]）
*{{lang|sco|Unitit Kinrick o Great Breetain an Northren Ireland}}（[[スコットランド語]]）
**{{lang|sco|Claught Kängrick o Docht Brätain an Norlin Airlann}}、{{lang|sco|Unitet Kängdom o Great Brittain an Norlin Airlann}}（アルスター・スコットランド語）</ref>
|国旗画像 = Flag of the United Kingdom.svg
|国章画像 = [[ファイル:Royal Coat of Arms of the United Kingdom.svg|85px|イギリスの国章]]
|国章リンク =（[[イギリスの国章|国章]]）
|標語 = {{lang|fr|[[Dieu et mon droit]]}}<br />（[[フランス語]]:[[Die

### 21. カテゴリ名を含む行を抽出


In [37]:
category = re.compile("\[\[Category:.*\]\]")
include_category_lines = [line for line in uk_wiki['text'].split("\n") if category.search(line)]
print("\n".join(include_category_lines[:50]))

[[Category:イギリス|*]]
[[Category:イギリス連邦加盟国]]
[[Category:英連邦王国|*]]
[[Category:G8加盟国]]
[[Category:欧州連合加盟国|元]]
[[Category:海洋国家]]
[[Category:現存する君主国]]
[[Category:島国]]
[[Category:1801年に成立した国家・領域]]


### 22. カテゴリ名の抽出


In [58]:
# ([^|\]]+)：カテゴリ名部分にマッチするキャプチャグループです。
# |または]のいずれにも一致しない文字が1文字以上続く部分を取得します。
# by ChatGPT
file_pattern = re.compile(r'\[\[Category:([^|\]]+)')

categories = []
for line in uk_wiki['text'].split("\n"):
    matches = file_pattern.findall(line)
    if len(matches) > 0:
        categories += matches

print(categories)

['イギリス', 'イギリス連邦加盟国', '英連邦王国', 'G8加盟国', '欧州連合加盟国', '海洋国家', '現存する君主国', '島国', '1801年に成立した国家・領域']


### 23. セクション構造


In [78]:
section_pattern = re.compile(r'^(=+)\s*(.*?)\s*\1$')

sections = []
for line in uk_wiki['text'].split("\n"):
    matches = section_pattern.findall(line)
    if len(matches) > 0:
        sections += [(len(level)-1, title) for level, title in matches]

print(sections[:10])

[(1, '国名'), (1, '歴史'), (1, '地理'), (2, '主要都市'), (2, '気候'), (1, '政治'), (2, '元首'), (2, '法'), (2, '内政'), (2, '地方行政区分')]


### 24. ファイル参照の抽出


In [80]:
file_pattern = re.compile(r'\[\[ファイル:([^|\]]+)')

files = []
for line in uk_wiki['text'].split("\n"):
    matches = file_pattern.findall(line)
    if len(matches) > 0:
        files += matches

print(files[:5])


['Royal Coat of Arms of the United Kingdom.svg', 'United States Navy Band - God Save the Queen.ogg', 'Descriptio Prime Tabulae Europae.jpg', "Lenepveu, Jeanne d'Arc au siège d'Orléans.jpg", 'London.bankofengland.arp.jpg']


### 25. テンプレートの抽出


In [201]:
def extract_jawiki_info(text: str, value_processes: list[callable] = None) -> dict:
    info_match = re.search(r'\{\{基礎情報.*?(\n.+?)\n\}\}', text, re.DOTALL)
    info_text = info_match.group(1)

    info_dict = {}
    map_pat = re.compile(r'(.+?)\s=\s(.*)', re.DOTALL)
    for kv in info_text.split("\n|"):
        matches = map_pat.findall(kv)
        if matches:
            key, val = matches[0]
            if value_processes is not None:
                for proc in value_processes:
                    val = proc(val)
            info_dict.update([(key, val)])
    return info_dict

info_dict = extract_jawiki_info(uk_wiki['text'])
list(info_dict.items())[:10]

[('日本語国名', 'グレートブリテン及び北アイルランド連合王国'),
 ('公式国名',
  '{{lang|en|United Kingdom of Great Britain and Northern Ireland}}<ref>英語以外での正式国名:<br />\n*{{lang|gd|An Rìoghachd Aonaichte na Breatainn Mhòr agus Eirinn mu Thuath}}（[[スコットランド・ゲール語]]）\n*{{lang|cy|Teyrnas Gyfunol Prydain Fawr a Gogledd Iwerddon}}（[[ウェールズ語]]）\n*{{lang|ga|Ríocht Aontaithe na Breataine Móire agus Tuaisceart na hÉireann}}（[[アイルランド語]]）\n*{{lang|kw|An Rywvaneth Unys a Vreten Veur hag Iwerdhon Glédh}}（[[コーンウォール語]]）\n*{{lang|sco|Unitit Kinrick o Great Breetain an Northren Ireland}}（[[スコットランド語]]）\n**{{lang|sco|Claught Kängrick o Docht Brätain an Norlin Airlann}}、{{lang|sco|Unitet Kängdom o Great Brittain an Norlin Airlann}}（アルスター・スコットランド語）</ref>'),
 ('国旗画像', 'Flag of the United Kingdom.svg'),
 ('国章画像',
  '[[ファイル:Royal Coat of Arms of the United Kingdom.svg|85px|イギリスの国章]]'),
 ('標語',
  '{{lang|fr|[[Dieu et mon droit]]}}<br />（[[フランス語]]:[[Dieu et mon droit|神と我が権利]]）'),
 ('国歌',
  "[[女王陛下万歳|{{lang|en|God Save the Queen}}]]{{en icon}}<br

### 26. 強調マークアップの除去


In [203]:
def removed_strong(text: str) -> str:
    ret = re.sub(r"''+(.*?)''+", r'\1', text)
    return ret

info_dict = extract_jawiki_info(uk_wiki['text'], [removed_strong])
list(info_dict.items())[:10]

[('日本語国名', 'グレートブリテン及び北アイルランド連合王国'),
 ('公式国名',
  '{{lang|en|United Kingdom of Great Britain and Northern Ireland}}<ref>英語以外での正式国名:<br />\n*{{lang|gd|An Rìoghachd Aonaichte na Breatainn Mhòr agus Eirinn mu Thuath}}（[[スコットランド・ゲール語]]）\n*{{lang|cy|Teyrnas Gyfunol Prydain Fawr a Gogledd Iwerddon}}（[[ウェールズ語]]）\n*{{lang|ga|Ríocht Aontaithe na Breataine Móire agus Tuaisceart na hÉireann}}（[[アイルランド語]]）\n*{{lang|kw|An Rywvaneth Unys a Vreten Veur hag Iwerdhon Glédh}}（[[コーンウォール語]]）\n*{{lang|sco|Unitit Kinrick o Great Breetain an Northren Ireland}}（[[スコットランド語]]）\n**{{lang|sco|Claught Kängrick o Docht Brätain an Norlin Airlann}}、{{lang|sco|Unitet Kängdom o Great Brittain an Norlin Airlann}}（アルスター・スコットランド語）</ref>'),
 ('国旗画像', 'Flag of the United Kingdom.svg'),
 ('国章画像',
  '[[ファイル:Royal Coat of Arms of the United Kingdom.svg|85px|イギリスの国章]]'),
 ('標語',
  '{{lang|fr|[[Dieu et mon droit]]}}<br />（[[フランス語]]:[[Dieu et mon droit|神と我が権利]]）'),
 ('国歌',
  '[[女王陛下万歳|{{lang|en|God Save the Queen}}]]{{en icon}}<br

### 27. 内部リンクの除去


In [245]:
text = "[[internal link|test]]"
def removed_internal_link(text: str) -> str:
    ret = re.sub(r"\[\[(.*)\]\]", r'\1', text)
    return ret
print(removed_internal_link(text))

processes = [removed_strong, removed_internal_link]
info_dict = extract_jawiki_info(uk_wiki['text'], processes)
list(info_dict.items())[:10]

internal link|test


[('日本語国名', 'グレートブリテン及び北アイルランド連合王国'),
 ('公式国名',
  '{{lang|en|United Kingdom of Great Britain and Northern Ireland}}<ref>英語以外での正式国名:<br />\n*{{lang|gd|An Rìoghachd Aonaichte na Breatainn Mhòr agus Eirinn mu Thuath}}（スコットランド・ゲール語）\n*{{lang|cy|Teyrnas Gyfunol Prydain Fawr a Gogledd Iwerddon}}（ウェールズ語）\n*{{lang|ga|Ríocht Aontaithe na Breataine Móire agus Tuaisceart na hÉireann}}（アイルランド語）\n*{{lang|kw|An Rywvaneth Unys a Vreten Veur hag Iwerdhon Glédh}}（コーンウォール語）\n*{{lang|sco|Unitit Kinrick o Great Breetain an Northren Ireland}}（スコットランド語）\n**{{lang|sco|Claught Kängrick o Docht Brätain an Norlin Airlann}}、{{lang|sco|Unitet Kängdom o Great Brittain an Norlin Airlann}}（アルスター・スコットランド語）</ref>'),
 ('国旗画像', 'Flag of the United Kingdom.svg'),
 ('国章画像', 'ファイル:Royal Coat of Arms of the United Kingdom.svg|85px|イギリスの国章'),
 ('標語',
  '{{lang|fr|Dieu et mon droit]]}}<br />（[[フランス語]]:[[Dieu et mon droit|神と我が権利）'),
 ('国歌',
  '女王陛下万歳|{{lang|en|God Save the Queen}}]]{{en icon}}<br />神よ女王を護り賜え<br />{{center|[[ファイ

### 28. MediaWiki マークアップの除去


### 国旗画像の URL を取得する


In [264]:
info_dict = extract_jawiki_info(uk_wiki['text'])
info_dict["国旗画像"]
import requests

session = requests.Session()

url = "https://www.mediawiki.org/w/api.php"

params = {
    "action": "query",
    "format": "json",
    "prop": "imageinfo",
    "titles": "File:Flag_of_the_United_Kingdom.svg",
    "iiprop": "url"
}

res = session.get(url=url, params=params)
data = res.json()
url = data['query']['pages']['-1']['imageinfo'][0]['url']
print("url: ", url)

url:  https://upload.wikimedia.org/wikipedia/commons/8/83/Flag_of_the_United_Kingdom_%283-5%29.svg
